In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
raw_df=pd.read_csv("../input/clothessizeprediction/final_test.csv")

In [ ]:
raw_df.head()

In [ ]:
raw_df.tail()

In [ ]:
raw_df.describe()

In [ ]:
raw_df.count()

In [ ]:
raw_df["height"].fillna(raw_df["height"].mean(),inplace=True)

In [ ]:
raw_df["age"].fillna(raw_df["age"].median(),inplace=True)

In [ ]:
raw_df.isnull().sum()

In [ ]:
raw_df.groupby("size")[["weight","height"]].mean()

In [ ]:
sns.heatmap(raw_df.corr(), annot=True, fmt='.1g', cmap="viridis",);

In [ ]:
raw_df["size"].value_counts()

In [ ]:
sns.countplot(x=raw_df["size"]);

In [ ]:
sns.distplot(raw_df["height"], color="blue");

In [ ]:
sns.distplot(raw_df["weight"], color="b");

In [ ]:
sns.distplot(raw_df["age"], color="b");

Outlier removal

In [ ]:
dfs = []
sizes = []
for size_type in raw_df['size'].unique():
    print('size type:',size_type)
    sizes.append(size_type)
    ndf = raw_df[['age','height','weight']][raw_df['size'] == size_type]
    zscore = ((ndf - ndf.mean())/ndf.std())
    dfs.append(zscore)

In [ ]:
for i in range(len(dfs)):
    print(sizes[i])
    dfs[i]['age'] = dfs[i]['age'][(dfs[i]['age']>-3) & (dfs[i]['age']<3)]
    dfs[i]['height'] = dfs[i]['height'][(dfs[i]['height']>-3) & (dfs[i]['height']<3)]
    dfs[i]['weight'] = dfs[i]['weight'][(dfs[i]['weight']>-3) & (dfs[i]['weight']<3)]

In [ ]:
for i in range(len(sizes)):
    dfs[i]['size'] = sizes[i]
df = pd.concat(dfs)
df.head()

In [ ]:
df['size'] = df['size'].map({'XXS': 1, 'S': 2, "M" : 3, "L" : 4, "XL" : 5, "XXL" : 6, "XXXL" : 7})

In [ ]:
df.head()

In [ ]:
df["height"].fillna(df["height"].mean(),inplace=True)
df["age"].fillna(df["age"].median(),inplace=True)
df["weight"].fillna(df["weight"].mean(),inplace=True)

In [ ]:
X = df.drop("size", axis=1)

In [ ]:
X.head()

In [ ]:
X["bmi"]=X["height"]/X["weight"]

In [ ]:
X.drop("height",axis=1)

In [ ]:
X.drop("weight",axis=1)

In [ ]:
Y = df["size"]
Y.head()

Splitting into Train and Test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1)

In [ ]:
len(X_train), len(X_test)

In [ ]:
transfer = StandardScaler()
X_train = transfer.fit_transform(X_train)
X_test = transfer.transform(X_test)

In [ ]:
df.isnull().sum()

**Method 1 - Linear Regression --> Bad Accuracy after Outlier removal**

In [ ]:
model = LinearRegression()
model.fit(X_train,Y_train)

In [ ]:
Y_pred=model.predict(X_test)
Y_pred

In [ ]:
LRScore = model.score(X_test,Y_test)
print("Accuracy for Method 1 - Linear Regression:",LRScore*100)

**Method 2 - Random Forest--> Bad Accuracy after Outlier removal**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier()

In [ ]:
model1.fit(X_train,Y_train)

In [ ]:
Y_pred_rf=model1.predict(X_test)
Y_pred_rf

In [ ]:
RandomForestClassifierScore = model.score(X_test,Y_test)
print("Accuracy obtained for Method 2 - Random Forest Classifier :",RandomForestClassifierScore*100)

In [ ]:
importance = model1.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

**Method 3 - Decision Tree--> Great Accuracy after Outlier removal**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

In [ ]:
tree.fit(X_train,Y_train)

In [ ]:
DecisionTreeClassifierScore = tree.score(X_test,Y_test)
print("Accuracy obtained by Decision Tree Classifier model:",DecisionTreeClassifierScore*100)

**Method 4 - XGBRF Classifier--> Good Accuracy after Outlier removal**

In [ ]:
from xgboost import XGBRFClassifier
modelXGBRF = XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.2)

In [ ]:
modelXGBRF.fit(X_train,Y_train)

In [ ]:
preds=modelXGBRF.predict(X_test)
preds

In [ ]:
XGBRFScore=modelXGBRF.score(X_test,Y_test)
print("Accuracy obtained by XGBRF model:",XGBRFScore*100)

**Method 5 - KNN--> Good Accuracy after Outlier removal**

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors=7, metric='manhattan', weights='distance')
KNN_model.fit(X_train, Y_train)

In [ ]:
KNN_model.score(X_test, Y_test)